In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
from sklearn.metrics import balanced_accuracy_score


Bad key text.latex.preview in file /home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # w

In [2]:
all_meta = pd.read_csv('data/train_metadata_extended.csv')
with open('data/scored_birds.json') as fin:
    test_birds = json.load(fin)

In [3]:
all_meta.loc[:, 'secondary_labels'] = all_meta.secondary_labels.apply(eval)
all_meta['target_raw'] = all_meta.secondary_labels + all_meta.primary_label.apply(lambda x: [x])

In [4]:
all_species = sorted(set(all_meta.target_raw.sum()))
species2id = {s: i for i, s in enumerate(all_species)}
id2species = {i: s for i, s in enumerate(all_species)}

all_meta['target'] = all_meta.target_raw.apply(lambda species: [int(s in species) for s in all_species])

In [42]:
last_pred = np.load('Lab02_NMT/last_pred.npy')
last_true = np.load('Lab02_NMT/last_true.npy')

In [43]:
test_idx = [species2id[b] for b in test_birds]
last_true[:, test_idx].sum(axis=0)

array([  4.,   6.,  25.,   3.,   0.,   7.,   1.,  13.,   0.,   2.,   1.,
         0.,  87.,  16.,  18.,   0.,   9.,   0., 107.,  22.,  10.],
      dtype=float32)

In [44]:
def score_pred(true, pred):
    if true.sum() == 0:
        return np.nan
    return balanced_accuracy_score(
            true,
            pred
        )

In [45]:
score_stat = []
trsh = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]

for t in trsh:
    score_stat.append({
        b: score_pred(
            last_true[:, species2id[b]],
            last_pred[:, species2id[b]] > t
        )
        for b in test_birds
    })
    
score_stat = pd.DataFrame(score_stat).T
score_stat.columns = trsh

In [46]:
score_stat

,0.01,0.05,0.10,0.15,0.20,0.25
akiapo,0.991405,0.995113,0.872809,0.748483,0.749157,0.749494
aniani,0.990725,0.830129,0.831816,0.832490,0.832996,0.832996
apapan,0.943340,0.963876,0.968968,0.951344,0.893041,0.874229
barpet,0.984501,0.664308,0.665487,0.666330,0.666498,0.666498
crehon,NaN,NaN,NaN,NaN,NaN,NaN
elepai,0.828128,0.843142,0.848202,0.851070,0.852420,0.853938
ercfra,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
hawama,0.963827,0.945649,0.911076,0.875488,0.876671,0.838378
hawcre,NaN,NaN,NaN,NaN,NaN,NaN
hawgoo,0.995958,0.749832,0.499832,0.499832,0.500000,0.500000


In [47]:
score_stat.idxmax(axis=1).fillna(0.05).to_dict()

{'akiapo': 0.05,
 'aniani': 0.01,
 'apapan': 0.1,
 'barpet': 0.01,
 'crehon': 0.05,
 'elepai': 0.25,
 'ercfra': 0.01,
 'hawama': 0.01,
 'hawcre': 0.05,
 'hawgoo': 0.01,
 'hawhaw': 0.01,
 'hawpet1': 0.05,
 'houfin': 0.05,
 'iiwi': 0.25,
 'jabwar': 0.01,
 'maupar': 0.05,
 'omao': 0.05,
 'puaioh': 0.05,
 'skylar': 0.05,
 'warwhe1': 0.01,
 'yefcan': 0.01}